# Add @corresp

In [11]:
%%time

import os
import xml.etree.ElementTree as ET
import requests
import json

TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"

def remove_ns_prefix(tree):
    root = tree.getroot()

    # Supprimer les préfixes mais conserver les espaces de noms
    for elem in root.iter():
        if '}' in elem.tag:
            elem.tag = elem.tag.split('}', 1)[1]  # Supprime le préfixe

    # Ajouter l'espace de noms TEI en tant que namespace par défaut
    ET.register_namespace('', TEI_NAMESPACE)  

    return tree

def fetch_corresp_value(key_value):
    """Appelle l'API pour récupérer la valeur de corresp en fonction du key."""
    
    if key_value.startswith("stutzmann_himanis#"):
        record_id = key_value.split("#")[-1]
        url = f"https://heurist.huma-num.fr/heurist/api/records?db=stutzmann_himanis&q=[{{\"ids\":\"{record_id}\"}}]"
        
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            if "records" in data and data["records"]:
                record = data["records"][0]
                fetched_id = record.get("rec_ID", "")
                corresp_value = record.get("rec_Title", "")
                
                return fetched_id, corresp_value
        except requests.RequestException as e:
            print(f"⚠️ Erreur API pour {key_value}: {e}")
    
    return None, ""

def process_xml_file(file_path):
    """Traite un fichier XML : supprime les préfixes et ajoute @corresp si nécessaire."""
    
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    # Ajout de l'attribut xmlns à l'élément racine <TEI> s'il n'est pas déjà présent
    if "xmlns" not in root.attrib:
        root.set("xmlns", TEI_NAMESPACE)
    
    elements_to_update = [
        elem for elem in root.iter()
        if 'key' in elem.attrib and elem.attrib['key'].startswith('stutzmann_himanis#') and 'corresp' not in elem.attrib
    ]
    
    # Si aucun élément n'est concerné, on ne modifie pas le fichier
    if not elements_to_update:
        return
    
    # Mise à jour des éléments avec @corresp
    for elem in elements_to_update:
        key_value = elem.attrib['key']
        record_id = key_value.split("#")[-1]  # ID actuel dans key
        print(record_id)
        fetched_id, corresp_value = fetch_corresp_value(key_value)
        
        if corresp_value:
            elem.set('corresp', corresp_value.replace("[", "(").replace("]", ")"))
        
        # Vérification de la correspondance de l'ID et mise à jour si nécessaire
        if fetched_id and fetched_id != record_id:
            elem.set('key', f"stutzmann_himanis#{fetched_id}")
    
    # Suppression des préfixes avant sauvegarde
    tree = remove_ns_prefix(tree)
    
    # Sauvegarde du fichier modifié
    tree.write(file_path, encoding='utf-8', xml_declaration=True)

def process_xml_folder(folder_path):
    """Parcourt un dossier et traite chaque fichier XML."""
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".xml"):
            file_path = os.path.join(folder_path, file_name)
            print(f"📂 Traitement du fichier : {file_name}")
            process_xml_file(file_path)

# Définition du dossier contenant les fichiers XML
dossier_xml = "../"  # À adapter selon l'emplacement réel des fichiers

# Traitement du dossier XML
process_xml_folder(dossier_xml)


📂 Traitement du fichier : biblio.xml
📂 Traitement du fichier : taxonomy.xml
📂 Traitement du fichier : x1117_d1e209543.xml
325243
📂 Traitement du fichier : x1131_d1e209711.xml
📂 Traitement du fichier : x1136_d1e209924.xml
📂 Traitement du fichier : x1136_d1e210024.xml
324643
282689
325031
324643
📂 Traitement du fichier : x1136_d1e210405.xml
282689
324533
324479
324911
324533
324533
324621
324487
282689
324479
282689
324487
324421
324533
282533
324487
324408
325264
282689
324487
324426
324443
282689
324533
282533
325179
324646
324643
324533
282533
324453
325251
282689
324479
325283
325277
325283
324949
324479
324479
324421
324455
324370
325273
⚠️ Erreur API pour stutzmann_himanis#325273: 404 Client Error: Not Found for url: https://heurist.huma-num.fr/heurist/api/records?db=stutzmann_himanis&q=%5B%7B%22ids%22:%22325273%22%7D%5D
324404
324459
324459
324356
324359
324356
282689
324479
324356
324455
325251
324455
324370
324356
282689
325283
324455
325251
325168
324356
324356
324442
324356
32